In [1]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import train_test_split, RandomizedSearchCV
# from sklearn.metrics import mean_squared_error, mean_absolute_error
# from sklearn.impute import SimpleImputer
# import warnings
# import joblib  # Sử dụng để lưu mô hình
# from sklearn.feature_selection import SelectKBest, f_regression

# # Tắt cảnh báo
# warnings.filterwarnings('ignore')

# # Đọc dữ liệu
# df = pd.read_csv('clean_data.csv')

# # Chuyển đổi cột Date thành kiểu datetime
# df['Date'] = pd.to_datetime(df['Date'])

# # Tạo các biến đặc trưng cho mô hình
# holiday_mapping = {
#     'Normal Day': 0,
#     'Super Bowl': 1,
#     'Labor Day': 2,
#     'Thanksgiving': 3,
#     'Christmas': 4
# }

# # Chuyển đổi IsHoliday thành số
# df['IsHoliday'] = df['IsHoliday'].map(holiday_mapping)

# # Chuyển đổi Type thành số nếu cần
# df['Type'] = df['Type'].map({'A': 1, 'B': 2, 'C': 3})

# # Tách năm, tháng và ngày từ cột Date
# df['Year'] = df['Date'].dt.year
# df['Month'] = df['Date'].dt.month
# df['Day'] = df['Date'].dt.day
# df['Week_Number'] = df['Date'].dt.isocalendar().week

# # Chọn các cột làm đặc trưng (X) và mục tiêu (y)
# X = df[['Store', 'Dept', 'IsHoliday', 'Temperature', 'Fuel_Price', 
#          'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 
#          'MarkDown5', 'CPI', 'Unemployment', 'Type', 'Size', 
#          'Year', 'Month', 'Day', 'Week_Number']]
# y = df['Weekly_Sales']

# # Giảm kích thước tập dữ liệu để thử nghiệm nhanh hơn
# X_sample, _, y_sample, _ = train_test_split(X, y, test_size=0.95, random_state=42)

# # Sử dụng SimpleImputer để xử lý các giá trị thiếu (nếu có)
# imputer = SimpleImputer(strategy='mean')
# X_sample = imputer.fit_transform(X_sample)

# # Chọn các đặc trưng quan trọng nhất (giảm số chiều dữ liệu)
# selector = SelectKBest(f_regression, k=10)
# X_sample = selector.fit_transform(X_sample, y_sample)

# # Định nghĩa các tham số để tìm kiếm với RandomizedSearchCV (giảm số lượng giá trị để thử)
# param_dist = {
#     'n_estimators': [50, 100],  # Giảm số lượng cây
#     'max_depth': [10, 20, None],
#     'min_samples_split': [5, 10],
#     'min_samples_leaf': [1, 2],
#     'bootstrap': [True]
# }

# # Khởi tạo mô hình Random Forest
# rf = RandomForestRegressor(random_state=42, n_jobs=-1)

# # Khởi tạo RandomizedSearchCV (giảm số lần chạy và tăng tốc)
# random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, 
#                                    n_iter=10, cv=3, scoring='neg_mean_squared_error', 
#                                    n_jobs=-1, verbose=1, random_state=42)

# # Fit mô hình trên tập mẫu nhỏ
# random_search.fit(X_sample, y_sample)

# # Lưu mô hình tốt nhất để sử dụng sau này
# best_rf = random_search.best_estimator_
# joblib.dump(best_rf, 'optimized_rf_model.pkl')

# # Áp dụng mô hình tốt nhất lên toàn bộ tập dữ liệu
# X = imputer.transform(X)  # Xử lý dữ liệu toàn bộ với imputer
# X = selector.transform(X)  # Chọn các đặc trưng quan trọng nhất
# y_pred_best = best_rf.predict(X)

# # Đánh giá mô hình
# mse_best = mean_squared_error(y, y_pred_best)
# mae_best = mean_absolute_error(y, y_pred_best)
# print(f'Optimized MSE: {mse_best}, MAE: {mae_best}')

# # Tạo DataFrame cho doanh số dự đoán (không thay đổi phần này)
# # Số tuần dự đoán thay đổi từ 20 xuống 8
# future_weeks = np.arange(len(df), len(df) + 8).reshape(-1, 1)
# future_data = pd.DataFrame({
#     'Store': X[:, 0].mode()[0],  # Sử dụng Store phổ biến nhất
#     'Dept': X[:, 1].mode()[0],    # Sử dụng Dept phổ biến nhất
#     'IsHoliday': 0,  # Giả sử không có ngày lễ
#     'Temperature': X[:, 3].mean(),  # Nhiệt độ trung bình
#     'Fuel_Price': X[:, 4].mean(),  # Giá nhiên liệu trung bình
#     'MarkDown1': 0,  # Giá Markdown trung bình hoặc 0
#     'MarkDown2': 0,
#     'MarkDown3': 0,
#     'MarkDown4': 0,
#     'MarkDown5': 0,
#     'CPI': X[:, 10].mean(),  # CPI trung bình
#     'Unemployment': X[:, 11].mean(),  # Tỷ lệ thất nghiệp trung bình
#     'Type': X[:, 12].mode()[0],  # Loại phổ biến nhất
#     'Size': X[:, 13].mean(),  # Kích thước trung bình
#     'Year': [2025] * 8,  # Dự đoán cho năm tiếp theo
#     'Month': np.tile(np.arange(1, 6), 4)[:8],  # Các tháng từ 1 đến 5
#     'Day': np.tile(np.arange(1, 6), 4)[:8],  # Các ngày từ 1 đến 5
#     'Week_Number': np.arange(1, 9)  # Số tuần từ 1 đến 8
# })

# # Dự đoán doanh số cho 8 tuần tiếp theo
# future_sales = best_rf.predict(future_data)

# # Tạo DataFrame cho doanh số dự đoán
# future_dates = pd.date_range(start=df['Date'].iloc[-1] + pd.Timedelta(weeks=1), periods=8, freq='W')
# future_df = pd.DataFrame({'Date': future_dates, 'Weekly_Sales': future_sales})

# # Vẽ biểu đồ
# plt.figure(figsize=(14, 7))
# plt.plot(df['Date'], df['Weekly_Sales'], color='blue', linestyle='-', label='Doanh số hiện tại')
# plt.plot(future_df['Date'], future_df['Weekly_Sales'], color='orange', linestyle='-', label='Dự đoán doanh số')
# plt.title('Dự đoán doanh số Weekly Sales cho 8 tuần tiếp theo')
# plt.xlabel('Ngày')
# plt.ylabel('Doanh số')
# plt.legend()
# plt.xticks(rotation=45)
# plt.tight_layout()
# plt.grid()
# plt.show()
